.

Data_engineering_TIL(20201012)

실습시 참고한 자료 : 패스트캠퍼스 데이터사이언스 스쿨 강의자료

** URL : https://www.fastcampus.co.kr/data_school_online

[Redis 개요]

데이터베이스, 캐시, 브로커로 사용되는 오픈소스 메모리 데이터 저장소

[실습내용]

- 사전작업

먼저 AWS에 접속해서 amazon linux ami 2로 ec2를 띄운다. 

그리고 보안그룹에서 6379포트를 해당ec2 퍼블릭 아이피에 대해 열어줘야 한다.

- Redis 설치

In [ ]:
[ec2-user@ip-10-1-10-184 ~]$ sudo yum update -y

[ec2-user@ip-10-1-10-184 ~]$ sudo yum install python3 -y

[ec2-user@ip-10-1-10-184 ~]$ sudo amazon-linux-extras install redis4.0 -y

[ec2-user@ip-10-1-10-184 ~]$ sudo service redis start
Redirecting to /bin/systemctl start redis.service

[ec2-user@ip-10-1-10-184 ~]$ sudo service redis status
Redirecting to /bin/systemctl status redis.service
● redis.service - Redis persistent key-value database
   Loaded: loaded (/usr/lib/systemd/system/redis.service; disabled; vendor preset: disabled)
  Drop-In: /etc/systemd/system/redis.service.d
           └─limit.conf
   Active: active (running) since Mon 2020-10-12 01:32:00 UTC; 2s ago
 Main PID: 12098 (redis-server)
   CGroup: /system.slice/redis.service
           └─12098 /usr/bin/redis-server 127.0.0.1:6379

Oct 12 01:32:00 ip-10-1-10-184.ap-northeast-2.compute.internal systemd[1]: Starting Redis persistent...
Oct 12 01:32:00 ip-10-1-10-184.ap-northeast-2.compute.internal systemd[1]: Started Redis persistent ...
Hint: Some lines were ellipsized, use -l to show in full.
    
[ec2-user@ip-10-1-10-184 ~]$ sudo vim /etc/redis.conf

# 외부에서 접속할수 있도록 수정
# bind 127.0.0.1 을 아래와 같이 변경
bind = 0.0.0.0

# systemctl에서 management를 할수 있도록 수정
# supervised no 를 아래와 같이 변경
supervised systemd

[ec2-user@ip-10-1-10-184 ~]$ sudo service redis restart
Redirecting to /bin/systemctl restart redis.service

[ec2-user@ip-10-1-10-184 ~]$ sudo service redis status
Redirecting to /bin/systemctl status redis.service
● redis.service - Redis persistent key-value database
   Loaded: loaded (/usr/lib/systemd/system/redis.service; disabled; vendor preset: disabled)
  Drop-In: /etc/systemd/system/redis.service.d
           └─limit.conf
   Active: active (running) since Mon 2020-10-12 01:41:02 UTC; 3s ago
  Process: 8544 ExecStop=/usr/libexec/redis-shutdown (code=exited, status=0/SUCCESS)
 Main PID: 8558 (redis-server)
   CGroup: /system.slice/redis.service
           └─8558 /usr/bin/redis-server 0.0.0.0:6379

Oct 12 01:41:02 ip-10-1-10-184.ap-northeast-2.compute.internal systemd[1]: Starting Redis persistent...
Oct 12 01:41:02 ip-10-1-10-184.ap-northeast-2.compute.internal systemd[1]: Started Redis persistent ...
Hint: Some lines were ellipsized, use -l to show in full.

[ec2-user@ip-10-1-10-184 ~]$ sudo pip3 install redis
WARNING: Running pip install with root privileges is generally not a good idea. Try `pip3 install --user` instead.
Collecting redis
  Downloading https://files.pythonhosted.org/packages/a7/7c/24fb0511df653cf1a5d938d8f5d19802a88cef255706fdda242ff97e91b7/redis-3.5.3-py2.py3-none-any.whl (72kB)
    100% |████████████████████████████████| 81kB 1.9MB/s
Installing collected packages: redis
Successfully installed redis-3.5.3

- Redis에 데이터 저장하고 불러오기

In [ ]:
[ec2-user@ip-10-1-10-184 ~]$ sudo vim redis_test.py
import redis

# client = redis.StrictRedis(host="[ec2 public ip]", port=6379, db=0)
client = redis.StrictRedis(host="3.34.253.94", port=6379, db=0)

print(client)

[ec2-user@ip-10-1-10-184 ~]$ python3 redis_test.py
Redis<ConnectionPool<Connection<host=3.34.253.94,port=6379,db=0>>>

[ec2-user@ip-10-1-10-184 ~]$ sudo vim redis_tes2.py
import redis

# client = redis.StrictRedis(host="[ec2 public ip]", port=6379, db=0)
client = redis.StrictRedis(host="3.34.253.94", port=6379, db=0)

# string data save
result = client.set("key", "value")
print(result)

# read data
result = client.get("key")
print(result)

# decoding data
result = result.decode("ascii")
print(result)

[ec2-user@ip-10-1-10-184 ~]$ python3 redis_test2.py
True
b'value'
value

[ec2-user@ip-10-1-10-184 ~]$ sudo vim redis_test3.py
import redis
import json

# client = redis.StrictRedis(host="[ec2 public ip]", port=6379, db=0)
client = redis.StrictRedis(host="3.34.253.94", port=6379, db=0)

# data save
data = {"phone_code":[82,81,86], "nation":{"korea":"82"}}
print(type(data), data)

result = client.set("data", str(data))
print(result)

# data read
result = client.get("data")
print(result)

result = result.decode("ascii")
print(type(result), result)

# change type to dict
result = json.loads(result.replace("'","\""))
print(type(result), result)

[ec2-user@ip-10-1-10-184 ~]$ python3 redis_test3.py
<class 'dict'> {'phone_code': [82, 81, 86], 'nation': {'korea': '82'}}
True
b"{'phone_code': [82, 81, 86], 'nation': {'korea': '82'}}"
<class 'str'> {'phone_code': [82, 81, 86], 'nation': {'korea': '82'}}
<class 'dict'> {'phone_code': [82, 81, 86], 'nation': {'korea': '82'}}